# Structs

* A *struct* assembles several values of assorted types together into a single value, so you can deal with them as a unit. 
* Given a struct, you can read and modify its individual components. 
* A struct can have methods associated with it that operate on its components.

## Named-Field Structs

In [45]:
use std::fmt::Debug;
use std::fmt::Formatter;

#[derive(Clone, PartialEq)]
pub struct Person {
    name: String,
    age: u8,
}

impl Person {
    pub fn new(name: &str, age: u8) -> Self {
        Person { name: name.into(), age }
    }

    pub fn name(&self) -> &str {
        &self.name
    }

    pub fn set_name(&mut self, name: &str) {
        self.name = name.into();
    }

    pub fn age(&self) -> u8 {
        self.age
    }

    pub fn set_age(&mut self, age: u8) {
        self.age = age;
    }
}

impl Debug for Person {
    fn fmt(&self, f: &mut Formatter<'_>) -> std::fmt::Result {
        write!(f, "Person: name: {}, age: {};", self.name, self.age)
    }
}

In [44]:
let person = Person {
    name: String::from("Alice"),
    age: 30,
};

println!("{}, age: {}", person.name, person.age);

let mut another_person = Person::new("Bob", 25);

println!("another_person: {:?}", another_person);

Alice, age: 30
another_person: Person: name: Bob, age: 25;


In [13]:
another_person.set_age(44);
another_person.age()

44

### Declaring a Struct

In [27]:
// A rectangle of eight-bit grayscale pixels.
struct GrayscaleMap {
    pixels: Vec<u8>,
    size: (usize, usize)
}

### Creating a Struct Value

In [28]:
let width = 800;
let height = 600;

let image = GrayscaleMap {
    pixels: vec![0; width * height],
    size: (width, height)
};

In [30]:
fn create_image(width: usize, height: usize) -> GrayscaleMap {
    let pixels: Vec<u8> = vec![0; width * height];
    let size = (width, height);

    GrayscaleMap{
        pixels,
        size,
    } // struct expression
}

### Accessing Struct Fields

In [34]:
let image = create_image(800, 600);

assert!(image.size == (800, 600));
assert!(image.pixels.len() == image.size.0 * image.size.1);

### Visibility of Struct

* Structs are private by default, visible only in the module where they’re declared. 
* You can make a struct visible outside its module by prefixing its definition with `pub`

In [6]:
// A rectangle of eight-bit grayscale pixels.
pub struct GrayscaleMap {
    pub pixels: Vec<u8>,
    pub size: (usize, usize)
}

## Tuple-Like Structs

* The second kind of struct type is called a tuple-like struct, because it resembles a tuple:

In [ ]:
struct BoundingBox(usize, usize);

impl BoundingBox {
    const SCREEN: BoundingBox = BoundingBox(800, 600); // associated consts

    fn width(&self) -> usize {
        self.0
    }

    fn height(&self) -> usize {
        self.1
    }
}

let image_bbox = BoundingBox(800, 600);

assert!(image_bbox.0 * image_bbox.1 == 480_000);
assert!(image_bbox.width() * image_bbox.height() == 480_000);

assert_eq!(BoundingBox::SCREEN.width(), 800);

* Individual elements of a tuple-like struct may be public or not:

In [12]:
pub struct BoundingBox(pub usize, pub usize);

### Tuple-Like Struct as Newtypes

* Tuple-like structs allow you to create types that behave like tuples but have a name and can implement traits
* This allows you to create newtypes for stricter type checking
* This is a workaround for the *orphan rule* - [New Type Pattern](https://doc.rust-lang.org/book/ch20-02-advanced-traits.html#using-the-newtype-pattern-to-implement-external-traits)

In [16]:
fn evil_calculate_distance(velocity: f64, time: f64) -> f64 {
    velocity * time
}

#[derive(Debug, Copy, Clone, PartialEq, PartialOrd)]
struct Distance(f64);
assert!(Distance(50.0) == Distance(50.0));
assert!(Distance(50.0) < Distance(51.0));

#[derive(Debug, Copy, Clone, PartialEq, PartialOrd)]
struct Velocity(f64);

#[derive(Debug, Copy, Clone, PartialEq, PartialOrd)]
struct Time(f64);

// A newtype for stricter type checking
fn calculate_distance(velocity: Velocity, time: Time) -> Distance {
    Distance(velocity.0 * time.0)
}

let velocity = Velocity(10.0);
let time = Time(5.0);
let distance = calculate_distance(velocity, time);

println!("Distance traveled: {} kilometers", distance.0);

Distance traveled: 50 kilometers


## Unit-Like Structs

* The third kind of struct is a little obscure: it declares a struct type with no elements at all:

In [36]:
struct Onesuch;

let o: Onesuch = Onesuch;

* This allows to create types that can act as tags or markers

## Defining Methods - `impl`

In [2]:
pub struct Queue {
    older: Vec<char>,
    younger: Vec<char>
}

impl Queue {
    pub fn push(&mut self, c: char) {
        self.younger.push(c);
    }

    pub fn pop(&mut self) -> Option<char> {
        if self.older.is_empty() {
            if self.younger.is_empty() {
                return None;
            }

            use std::mem::swap;
            swap(&mut self.older, &mut self.younger);
            self.older.reverse();
        }

        self.older.pop()
    }

    pub fn is_empty(&self) -> bool {
        self.older.is_empty() && self.younger.is_empty()
    }
}

In [3]:
let mut q = Queue{ older: Vec::new(), younger: Vec::new() };

q.push('@');
q.push('$');
assert!(q.pop() == Some('@'));

q.push('*');
assert!(q.pop() == Some('$'));
assert!(q.pop() == Some('*'));
assert!(q.pop() == None);

assert!(q.is_empty());

In [4]:
impl Queue {
    pub fn split(self) -> (Vec<char>, Vec<char>) {
        (self.older, self.younger)
    }
}

In [5]:
let mut q = Queue { older: Vec::new(), younger: Vec::new() };
q.push('Q');
q.push('U');
assert!(q.pop() == Some('Q'));
q.push('X');

let (older, younger) = q.split();

assert_eq!(older, vec!['U']);
assert_eq!(younger, vec!['X']);

### `new` Method

* Constructor function - doesn't take `self` as an argument (**static function**)

In [15]:
impl Queue {
    pub fn new() -> Self {
        Queue { older: Vec::new(), younger: Vec::new() }
    }
}

In [24]:
let mut q = Queue::new();

q.push('a');
q.push('b');
q.push('c');

assert_eq!(q.pop(), Some('a'));
assert_eq!(q.pop(), Some('b'));
assert_eq!(q.pop(), Some('c'));
assert_eq!(q.pop(), None);

## Generic Structs

* Generics allow you to write flexible, reusable functions and types that can work with any data type. Instead of specifying concrete types, you use type parameters, which are placeholders for types that will be specified when the function or type is used.
* Generics use parameters passed in angle brackets to specify the types they work with. For example, `fn example<T>(param: T) {}` uses `T` as a generic type parameter.

In [17]:
pub struct Stack<T> {
    items: Vec<T>
}

impl<T> Stack<T> {
    pub fn new() -> Stack<T> {
        Stack{ items: Vec::new() }
    }

    pub fn push(&mut self, item: T) {
        self.items.push(item)
    }

    pub fn pop(&mut self) -> Option<T> {
        self.items.pop()
    }

    pub fn is_empty(&self) -> bool {
        self.items.is_empty()
    }
}

* For static method calls, you can supply the type parameter explicitly using the turbofish `::<>` notation:

In [19]:
let s = Stack::<char>::new();

* But in practice, Rust can figure it out for you:

In [21]:
let mut stack_1: Stack<&str> = Stack::new();
let mut stack_2: Stack<f64> = Stack::new();

stack_1.push("Hello");
stack_2.push(3.1415);

stack_1.push("World");
stack_2.push(2.71);

## Deriving Common Traits for Structs

* Structs are not copyable or comparable by default
* You can derive the `Copy` and `Clone` traits for a struct type, but only if all its fields are copyable or clonable
* You can derive the `PartialEq` and `Eq` traits for a struct type, but only if all its fields are comparable for equality
* You can derive `Debug` for a struct type, but only if all its fields are printable

In [47]:
#[derive(Copy, Clone, Debug, PartialEq, Default)]
struct Point {
    x: f64,
    y: f64
}

In [48]:
let mut pt = Point{ x: 0.0, y: 1.0 };

let other_pt = pt;
assert!(pt == other_pt);

pt.x = 42.42;
assert!(pt != other_pt);

println!("pt = {:?}", pt);
println!("other_pt = {:?}", other_pt);

pt = Point { x: 42.42, y: 1.0 }
other_pt = Point { x: 0.0, y: 1.0 }


In [49]:
let pt_default: Point = Point::default();

assert!(pt_default == Point { x: 0.0, y: 0.0 });

In [50]:
#[derive(Copy, Clone, Debug, PartialEq, Default)]
struct Rectangle {
    top_left: Point,
    width: u32,
    height: u32,
}

In [56]:
let mut rect = Rectangle::default();
println!("rect = {:?}", rect);

let rect2 = Rectangle {
    width: 100,
    height: 50,
    ..Default::default()  // rest syntax - rest will be filled with default values  
};
println!("rect2 = {:?}", rect2);

let rect3 = Rectangle {
    top_left: Point { x: 10.0, y: 20.0 },
    ..rect2  // rest syntax - rest will be filled with values from rect2
};
println!("rect3 = {:?}", rect3);

rect = Rectangle { top_left: Point { x: 0.0, y: 0.0 }, width: 0, height: 0 }
rect2 = Rectangle { top_left: Point { x: 0.0, y: 0.0 }, width: 100, height: 50 }
rect3 = Rectangle { top_left: Point { x: 10.0, y: 20.0 }, width: 100, height: 50 }


# Builder Pattern

In [68]:
#[derive(Debug, Clone, PartialEq, Eq)]
pub struct Config {
    host: String,
    port: u16,
    use_ssl: bool,
}

struct ConfigBuilder {
    host: Option<String>,
    port: Option<u16>,
    use_ssl: Option<bool>,
}

impl ConfigBuilder {
    pub fn new() -> Self {
        ConfigBuilder {
            host: None,
            port: None,
            use_ssl: None,
        }
    }

    fn host(mut self, host: &str) -> Self {
        self.host = Some(host.to_string());
        self
    }

    fn port(mut self, port: u16) -> Self {
        self.port = Some(port);
        self
    }

    fn use_ssl(mut self, use_ssl: bool) -> Self {
        self.use_ssl = Some(use_ssl);
        self
    }

    fn build(self) -> Config {
        Config {
            host: self.host.unwrap_or("localhost".to_string()),
            port: self.port.unwrap_or(8080),
            use_ssl: self.use_ssl.unwrap_or(false),
        }
    }
}

In [77]:
let config_1: Config = ConfigBuilder::new()
    .host("example.com")
    .port(443)
    .use_ssl(true)
    .build();

println!("config_1: {:?}", config_1);

config_1: Config { host: "example.com", port: 443, use_ssl: true }


In [78]:
let config_2 = ConfigBuilder::new()
    .host("example.org")
    .build();

println!("config_2: {:?}", config_2);

config_2: Config { host: "example.org", port: 8080, use_ssl: false }


# Destructuring Structs

In [86]:
let Config{host: host_name, port, ..} = ConfigBuilder::new().build();

In [88]:
host_name

"localhost"

In [89]:
port

8080

# PartialEq vs Eq

In [32]:
#[derive(Debug, Copy, Clone, PartialEq, Eq)]
struct Point {
    x: i32,
    y: i32,
}

In [33]:
#[derive(Debug, Clone, PartialEq, Eq, PartialOrd, Ord)]
struct Person {
    name: String,
    age: u8,
}

In [34]:
// let mut data = vec![3, 2, 1, 1, -5];
let mut data = vec![3.14, 2.71, f64::NAN, 1.73, -0.5];

//data.sort(); // will not compile because of not implemented Ord for f64
data.sort_by(|a, b| a.total_cmp(b));
data

[-0.5, 1.73, 2.71, 3.14, NaN]